# Import Libraries

In [1]:
from swarmintelligence import InspiredGreyWolfOptimizer, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))
rgb_misc_dataset = dict(itertools.islice(rgb_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
# optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
# optimal_params

In [5]:
optimal_params = {'wolfSize': 25, 'maxIteration': 100}

# Evaluation

In [6]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
# n_runs = 30
n_runs = 1

In [7]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = InspiredGreyWolfOptimizer(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g =InspiredGreyWolfOptimizer(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = InspiredGreyWolfOptimizer(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
308.8368958815901  |  1553.3863735614054  |  1043.1750398498077
Mean Fitness = 968.4661030976011
	 AirplaneF16.tiff | Threshold =  3
658.5029725341494  |  1791.276372008102  |  967.8432071605611
Mean Fitness = 1139.2075172342709
	 AirplaneF16.tiff | Threshold =  4
493.88519354137907  |  2276.3174586699943  |  1447.0955572842017
Mean Fitness = 1405.7660698318584
	 AirplaneF16.tiff | Threshold =  5
549.0333408704923  |  2341.495721964908  |  1188.000652657041
Mean Fitness = 1359.509905164147
	 Lena.png | Threshold =  2
758.2085732873654  |  1852.1309591136824  |  209.53180637051202
Mean Fitness = 939.9571129238534
	 Lena.png | Threshold =  3
580.0527582737674  |  2084.3047014020694  |  505.07752191484974
Mean Fitness = 1056.4783271968954
	 Lena.png | Threshold =  4
824.5041706221787  |  2283.026361210078  |  1469.4618299922386
Mean Fitness = 1525.6641206081651
	 Lena.png | Threshold =  5
877.8704094147251  |  2367.1868649377693  |  1602.5585234957068
M

## Visualize Results using DataFrame

In [8]:
gwo_evaluation_results_df = pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[49, 96]","[47, 90]","[39, 87]",308.836896,1553.386374,1043.175040,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",[308.8368958815901],[1553.3863735614054],[1043.1750398498077],[3.1749773025512695],[4849.729733784993],[69.64000096054704],[11.27362823996844],[0.600330992680007],[0.8910218534532147]
1,AirplaneF16.tiff,3,otsu,max,"[34, 59, 93]","[42, 67, 99]","[45, 77, 141]",658.502973,1791.276372,967.843207,...,"[[141, 255, 255, 255, 255, 255, 255, 255, 255,...",[658.5029725341494],[1791.276372008102],[967.8432071605611],[1.4401803016662598],[4361.7357177734375],[66.04343811290745],[11.734210131946467],[0.6378446841319265],[0.9016223265391142]
2,AirplaneF16.tiff,4,otsu,max,"[37, 54, 83, 113]","[18, 50, 89, 125]","[55, 83, 89, 106]",493.885194,2276.317459,1447.095557,...,"[[106, 255, 255, 255, 255, 255, 255, 255, 255,...",[493.88519354137907],[2276.3174586699943],[1447.0955572842017],[1.415881872177124],[3613.202101389567],[60.10991683066586],[12.551881067276447],[0.669118627082701],[0.9208875008550161]
3,AirplaneF16.tiff,5,otsu,max,"[26, 50, 71, 75, 101]","[15, 41, 61, 88, 130]","[71, 77, 77, 86, 119]",549.033341,2341.495722,1188.000653,...,"[[119, 255, 255, 255, 255, 255, 255, 255, 255,...",[549.0333408704923],[2341.495721964908],[1188.000652657041],[1.7969675064086914],[3760.88153330485],[61.3260265572852],[12.377907073823113],[0.6655223445247158],[0.9174734206109211]
4,Lena.png,2,otsu,max,"[54, 82]","[33, 74]","[55, 101]",758.208573,1852.130959,209.531806,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",[758.2085732873654],[1852.1309591136824],[209.53180637051202],[1.5412111282348633],[8933.882597605387],[94.51921813898689],[8.62040119867434],[0.45551272738183446],[0.756087113429771]
5,Lena.png,3,otsu,max,"[54, 63, 89]","[24, 47, 89]","[23, 44, 84]",580.052758,2084.304701,505.077522,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",[580.0527582737674],[2084.3047014020694],[505.07752191484974],[1.5541954040527344],[9374.743698120117],[96.82326010892278],[8.411209577496166],[0.45938672594499524],[0.7523999089469792]
6,Lena.png,4,otsu,max,"[54, 54, 57, 112]","[19, 31, 44, 110]","[52, 72, 89, 101]",824.504171,2283.026361,1469.461830,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",[824.5041706221787],[2283.026361210078],[1469.4618299922386],[1.8911166191101074],[5719.074977874756],[75.62456596817437],[10.55754570622371],[0.5399129030561436],[0.8357232769908017]
7,Lena.png,5,otsu,max,"[54, 54, 78, 81, 120]","[19, 45, 77, 78, 109]","[40, 45, 60, 85, 108]",877.870409,2367.186865,1602.558523,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",[877.8704094147251],[2367.1868649377693],[1602.5585234957068],[1.847764015197754],[5015.067548116048],[70.81714162627611],[11.128035739448734],[0.5641557402714511],[0.8573957066923376]


# Save results

In [9]:
# pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_otsu_RGB_df.pkl', 'wb'))
# pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_otsu_RGB_dict.pkl', 'wb'))